## Hugging Face Transformers Tutorial

### Setup

Create a venv and install requirements.txt

Package Imports

In [8]:
import torch
from torch import nn
from transformers import pipeline, AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AutoConfig, DataCollatorWithPadding
from datasets import load_dataset

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Examples

Sentiment Analysis

In [9]:
classifier = pipeline("sentiment-analysis")
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309


Multilingual Sentiment Analysis With AutoClass

In [18]:
from transformers import AutoModelForSequenceClassification

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name, torch_dtype="auto", device_map=device)
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device)

encoding = tokenizer("We are very happy to show you the 🤗 Transformers library.")
print(encoding)

classifier = pipeline("sentiment-analysis", model=pt_model, tokenizer=tokenizer)
classifier("Nous sommes très heureux de vous présenter la bibliothèque 🤗 Transformers.")

Device set to use cuda


{'input_ids': [101, 11312, 10320, 12495, 19308, 10114, 11391, 10855, 10103, 100, 58263, 13299, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


[{'label': '5 stars', 'score': 0.7272651791572571}]

In [20]:
pt_batch = tokenizer(
    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",
).to(device)
pt_outputs = pt_model(**pt_batch)
pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=-1)
print(pt_predictions)


tensor([[0.0021, 0.0018, 0.0115, 0.2121, 0.7725],
        [0.2084, 0.1826, 0.1969, 0.1755, 0.2365]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)


Saving and Loading Models

In [21]:
pt_save_directory = "./checkpoints/pt_save_pretrained"
tokenizer.save_pretrained(pt_save_directory)
pt_model.save_pretrained(pt_save_directory)

tokenizer = AutoTokenizer.from_pretrained(pt_save_directory)
pt_model = AutoModelForSequenceClassification.from_pretrained("./checkpoints/pt_save_pretrained")


Custom Model Build

In [22]:
my_config = AutoConfig.from_pretrained("distilbert/distilbert-base-uncased", n_heads=12)
my_model = AutoModel.from_config(my_config)

Training

In [23]:
from transformers import AutoModelForSequenceClassification, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", torch_dtype="auto")

training_args = TrainingArguments(
    output_dir="checkpoints/",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
)

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
dataset = load_dataset("rotten_tomatoes")

def tokenize_dataset(dataset):
    return tokenizer(dataset["text"])

dataset = dataset.map(tokenize_dataset, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 1066/1066 [00:00<00:00, 11891.20 examples/s]


Step,Training Loss
500,0.448300
1000,0.385600
1500,0.254900
2000,0.274900


TrainOutput(global_step=2134, training_loss=0.3359235246938975, metrics={'train_runtime': 75.3691, 'train_samples_per_second': 226.353, 'train_steps_per_second': 28.314, 'total_flos': 195974132394480.0, 'train_loss': 0.3359235246938975, 'epoch': 2.0})

Chat with the model

In [25]:
!transformers-cli chat --model_name_or_path Qwen/Qwen2.5-0.5B-Instruct

tokenizer_config.json: 100%|███████████████| 7.30k/7.30k [00:00<00:00, 19.1MB/s]
vocab.json: 100%|██████████████████████████| 2.78M/2.78M [00:01<00:00, 2.22MB/s]
merges.txt: 100%|██████████████████████████| 1.67M/1.67M [00:00<00:00, 1.70MB/s]
tokenizer.json: 100%|██████████████████████| 7.03M/7.03M [00:03<00:00, 1.84MB/s]
config.json: 100%|█████████████████████████████| 659/659 [00:00<00:00, 7.70MB/s]
model.safetensors: 100%|█████████████████████| 988M/988M [00:36<00:00, 27.1MB/s]
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
generation_config.json: 100%|███████████████████| 242/242 [00:00<00:00, 637kB/s]
<coder>:
^C


Gradio demos

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline("image-classification", model="google/vit-base-patch16-224")

gr.Interface.from_pipeline(pipe).launch()